<div><img  src="https://storage.googleapis.com/kaggle-organizations/3800/thumbnail.jpeg" style="width: 150px; 
    float: left; 
    margin-right: 10px;"></div><div  style="float: left; 
    width: calc(100% - 350px - 10px); font-size:30px; font-weight:bolder; letter-spacing:0.2em; text-transform:uppercase;" >Excellence in Research Award (Phase II)</div>

# Step 1. Import libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objects as go
import plotly
plotly.offline.init_notebook_mode(connected=True)
import geopandas as gpd
import warnings
!pip install openpyxl
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Step 2. Reading data into dataframe

In [ ]:
df_19=pd.read_excel("../input/phase-ii-widsdatathon2022/epa/epa/Datathon_EPA_Air_Quality_Demographics_Meteorology_2019.xlsx", parse_dates=True)

In [ ]:
df_20=pd.read_excel("../input/phase-ii-widsdatathon2022/epa/epa/Datathon_EPA_Air_Quality_Demographics_Meteorology_2020.xlsx", parse_dates=True)

# Step 3. Familiarity in data

In [ ]:
df_19.info()

In [ ]:
df_20.info()

In [ ]:
df_19.head()

In [ ]:
df_20.head()

# Step 4. Drop duplicates

In [ ]:
df_19.drop_duplicates(inplace=True)

In [ ]:
df_20.drop_duplicates(inplace=True)

# Step 5. Visualize missing values (NaN) values

In [ ]:
df_19[df_19['DATE'].isna()].head()

In [ ]:
fig = px.histogram(x = df_19.columns, y = df_19.isna().mean())

fig.update_layout(
        xaxis_title = "Columns",
        yaxis_title = "Missing Counts",
        title = "Total missings per columns", 
        plot_bgcolor = "#ECECEC")  

fig.show()

In [ ]:
fig = px.histogram(x = df_20.columns, y = df_20.isna().mean())

fig.update_layout(
        xaxis_title = "Columns",
        yaxis_title = "Missing Counts",
        title = "Total missings per columns", 
        plot_bgcolor = "#ECECEC")  

fig.show()

In [ ]:
fig = px.pie(df_19, values=df_19.isna().mean(), names=df_19.columns, title='Pie of missings in 2019')
fig.show()

In [ ]:
fig = px.pie(df_20, values=df_20.isna().mean(), names=df_20.columns, title='Pie of missings in 2022')
fig.show()

In [ ]:
s=0
for i in df_19.isna().sum():
    s+=i
print("Pie of missings in data at all", s/(129470*22))


# Step 6. Concat dataframes 19 and 20

In [ ]:
df_new=pd.concat([df_19,df_20])
df_new.info()

In [ ]:
fig = px.histogram(x = df_new.columns, y = df_new.isna().mean())

fig.update_layout(
        xaxis_title = "Columns",
        yaxis_title = "Missing Counts",
        title = "Total missings per columns", 
        plot_bgcolor = "#ECECEC")  

fig.show()

# Step 7. Drop missings from data

**Let's remove columns where missing values over 80%.**

In [ ]:
df_new1 = df_new.loc[:, df_new.isnull().mean() < .8]
df_new1.info()

# Step 8. Categorial and numerical columns 

In [ ]:
cat=[]
num=[]
for i in df_new.columns:
    if df_new[i].dtype=='object':
        cat.append(i)
    elif df_new[i].dtype=='float64':
        num.append(i)
        

#### Now we have two lists:
* **cat**-where we saved column names with categorical data
* **num**-where we saved column names with numerical data

In [ ]:
df_new[cat].info()

In [ ]:
df_new[num].info()

### Step 10. Analysis data with missings

In [ ]:
fig = px.line(a, x=df_new.DATE, y=df_new.BENZENE_PPBC,color=df_new.DATE.dt.year,
              title='BENZENE_PPBC')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

In [ ]:
fig = px.line(a, x=df_new.DATE, y=df_new.PM25_UG_PER_CUBIC_METER,color=df_new.DATE.dt.year,
              title='PM25_UG_PER_CUBIC_METER')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

# Step 9. Filling missing values

### Step 9.1. Filling numerical columns 

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer=IterativeImputer()
df_new[num] = imputer.fit_transform(df_new[num])
df_new.head()

In [ ]:
df_new.isnull().sum()

### Step 9.2. Filling categorical columns

In [ ]:

fig = px.scatter(df_new, x="BENZENE_PPBC", y="PM25_UG_PER_CUBIC_METER", facet_col=df_new.DATE.dt.year, color=df_new.DATE.dt.year, trendline="ols")
fig.show()

results = px.get_trendline_results(fig)
print(results)


In [ ]:
df_new['CBSA'].describe()[2]

In [ ]:
df_new['CBSA'].fillna('San Francisco-Oakland-Hayward, CA',inplace=True)

In [ ]:
df_new.isnull().sum()

#### Now we have data without missings.

# Step 10. Describe data

## Let's look for the data.

In [ ]:
df_new.describe().T

In [ ]:
df_new.describe(include='all').T

# Step 12.Correlation 

In [ ]:
fig = px.imshow(df_new.corr(), text_auto=True)
fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
)

fig.show()

In [ ]:
x=[]
for k,v in dict(df_new.corrwith(df_new["PM25_UG_PER_CUBIC_METER"])).items():
    if abs(v)>0.1 and abs(v)<0.9:
        x.append(k)
fig = px.imshow(df_new[x[1:]].corr(), text_auto=True)
fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
)

fig.show()
print(x)

# Step 11. Analysis of data

In [ ]:
df_new.groupby('COUNTY').agg({'PM25_UG_PER_CUBIC_METER': ['min', 'max'],'OZONE_PPM':['min','max'],'NO2_PPB':['min','max'],'CO_PPM':['min','max'],'SO2_PPB':['min','max']})

In [ ]:
df_new.columns

In [ ]:
df_new['year'] = pd.DatetimeIndex(df_new['DATE']).year

df_new[['STATE','CBSA',
       'PEOPLE_OF_COLOR_FRACTION', 'LOW_INCOME_FRACTION',
       'LINGUISTICALLY_ISOLATED_FRACTION', 'LESS_THAN_HS_ED_FRACTION', 'DATE',
       'TEMPERATURE_CELSIUS', 'RELATIVE_HUMIDITY',
       'WIND_SPEED_METERS_PER_SECOND', 'WIND_DIRECTION',
       'PM25_UG_PER_CUBIC_METER', 'OZONE_PPM', 'NO2_PPB', 'CO_PPM', 'SO2_PPB',
       'year']].groupby(['year','STATE']).agg('mean')

In [ ]:
df_new.STATE.unique()

In [ ]:
px.histogram(df_new.groupby('year').agg({'PM25_UG_PER_CUBIC_METER': ['mean'],'OZONE_PPM':['mean']}), color="year", barmode="overlay")

In [ ]:
df_new.groupby('STATE').agg({'PM25_UG_PER_CUBIC_METER': ['min', 'max'],'OZONE_PPM':['min','max'],'NO2_PPB':['min','max'],'CO_PPM':['min','max'],'SO2_PPB':['min','max']})

In [ ]:
df_new.groupby('STATE').agg({'PM25_UG_PER_CUBIC_METER': ['sum'],'OZONE_PPM':['sum'],'NO2_PPB':['sum'],'CO_PPM':['sum'],'SO2_PPB':['sum']})

In [ ]:
df_new.groupby('COUNTY').agg({'PM25_UG_PER_CUBIC_METER': ['mean'],'OZONE_PPM':['mean'],'NO2_PPB':['mean'],'CO_PPM':['mean'],'SO2_PPB':['mean']})

In [ ]:
df_new[num].columns

In [ ]:
sns.set_theme(style="whitegrid")
tips = sns.load_dataset("tips")
sns.set(rc={'figure.figsize':(15,10)})
sns.boxplot(df_19['PEOPLE_OF_COLOR_FRACTION'])

In [ ]:
sns.boxplot(df_19['LOW_INCOME_FRACTION'])

In [ ]:
sns.boxplot(df_19['LINGUISTICALLY_ISOLATED_FRACTION']) 

In [ ]:
#'LESS_THAN_HS_ED_FRACTION', 'TEMPERATURE_CELSIUS', 'RELATIVE_HUMIDITY',
  #     'WIND_SPEED_METERS_PER_SECOND', 'WIND_DIRECTION',
 #'PM25_UG_PER_CUBIC_METER', 'OZONE_PPM', 'NO2_PPB', 'CO_PPM', 'SO2_PPB'
sns.boxplot(df_19['LESS_THAN_HS_ED_FRACTION']) 

In [ ]:
#'LESS_THAN_HS_ED_FRACTION', 'TEMPERATURE_CELSIUS', 'RELATIVE_HUMIDITY',
  #     'WIND_SPEED_METERS_PER_SECOND', 'WIND_DIRECTION',
 #'PM25_UG_PER_CUBIC_METER', 'OZONE_PPM', 'NO2_PPB', 'CO_PPM', 'SO2_PPB'
sns.boxplot(df_19['TEMPERATURE_CELSIUS']) 

In [ ]:
#'LESS_THAN_HS_ED_FRACTION', 'TEMPERATURE_CELSIUS', 'RELATIVE_HUMIDITY',
  #     'WIND_SPEED_METERS_PER_SECOND', 'WIND_DIRECTION',
 #'PM25_UG_PER_CUBIC_METER', 'OZONE_PPM', 'NO2_PPB', 'CO_PPM', 'SO2_PPB'
sns.boxplot(df_19['RELATIVE_HUMIDITY']) 

In [ ]:
#'LESS_THAN_HS_ED_FRACTION', 'TEMPERATURE_CELSIUS', 'RELATIVE_HUMIDITY',
  #     'WIND_SPEED_METERS_PER_SECOND', 'WIND_DIRECTION',
 #'PM25_UG_PER_CUBIC_METER', 'OZONE_PPM', 'NO2_PPB', 'CO_PPM', 'SO2_PPB'
sns.boxplot(df_19['WIND_SPEED_METERS_PER_SECOND']) 

In [ ]:
#'LESS_THAN_HS_ED_FRACTION', 'TEMPERATURE_CELSIUS', 'RELATIVE_HUMIDITY',
  #     'WIND_SPEED_METERS_PER_SECOND', 'WIND_DIRECTION',
 #'PM25_UG_PER_CUBIC_METER', 'OZONE_PPM', 'NO2_PPB', 'CO_PPM', 'SO2_PPB'
sns.boxplot(df_19['WIND_DIRECTION']) 

In [ ]:
#'LESS_THAN_HS_ED_FRACTION', 'TEMPERATURE_CELSIUS', 'RELATIVE_HUMIDITY',
  #     'WIND_SPEED_METERS_PER_SECOND', 'WIND_DIRECTION',
 #'PM25_UG_PER_CUBIC_METER', 'OZONE_PPM', 'NO2_PPB', 'CO_PPM', 'SO2_PPB'
sns.boxplot(df_19['PM25_UG_PER_CUBIC_METER']) 

In [ ]:
#'LESS_THAN_HS_ED_FRACTION', 'TEMPERATURE_CELSIUS', 'RELATIVE_HUMIDITY',
  #     'WIND_SPEED_METERS_PER_SECOND', 'WIND_DIRECTION',
 #'PM25_UG_PER_CUBIC_METER', 'OZONE_PPM', 'NO2_PPB', 'CO_PPM', 'SO2_PPB'
sns.boxplot(df_19['OZONE_PPM']) 

In [ ]:
#'LESS_THAN_HS_ED_FRACTION', 'TEMPERATURE_CELSIUS', 'RELATIVE_HUMIDITY',
  #     'WIND_SPEED_METERS_PER_SECOND', 'WIND_DIRECTION',
 #'PM25_UG_PER_CUBIC_METER', 'OZONE_PPM', 'NO2_PPB', 'CO_PPM', 'SO2_PPB'
sns.boxplot(df_19['NO2_PPB']) 

In [ ]:
#'LESS_THAN_HS_ED_FRACTION', 'TEMPERATURE_CELSIUS', 'RELATIVE_HUMIDITY',
  #     'WIND_SPEED_METERS_PER_SECOND', 'WIND_DIRECTION',
 #'PM25_UG_PER_CUBIC_METER', 'OZONE_PPM', 'NO2_PPB', 'CO_PPM', 'SO2_PPB'
sns.boxplot(df_19['CO_PPM']) 

In [ ]:
#'LESS_THAN_HS_ED_FRACTION', 'TEMPERATURE_CELSIUS', 'RELATIVE_HUMIDITY',
  #     'WIND_SPEED_METERS_PER_SECOND', 'WIND_DIRECTION',
 #'PM25_UG_PER_CUBIC_METER', 'OZONE_PPM', 'NO2_PPB', 'CO_PPM', 'SO2_PPB'
sns.boxplot(df_19['SO2_PPB']) 

In [ ]:
data =  gpd.GeoDataFrame( df_new, geometry=gpd.points_from_xy(df_new.LONGITUDE, df_new.LATITUDE))